In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch

from tqdm import tqdm

import ipywidgets as widgets
from src.models import ContextSpaceNet

In [2]:
train_steps = 10000

n_in = 3 # 2 position + 1 context
n_out = 100 # no. of spatial representations
bs = 512 # batch size

model = ContextSpaceNet(n_in, n_out)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

loss_history = []
progress = tqdm(range(train_steps))
for i in progress: # train loop
    r = torch.tensor(np.random.uniform(-1, 1, (bs, 2)), dtype= torch.float32)
    c = torch.tensor(np.random.uniform(-2, 2, bs), dtype = torch.float32)
    inputs = torch.cat((r, c[:,None]), dim = -1)
    loss = model.train_step(x = inputs, y = inputs, optimizer = optimizer)

    if i % 10 == 0:
        loss_history.append(loss)
        progress.set_description(f"loss: {loss:>7f}")

loss: 0.000902: 100%|██████████| 10000/10000 [01:04<00:00, 155.24it/s]


In [ ]:
plt.semilogy(np.arange(0, train_steps, 10), loss_history, "-k", linewidth = 0.7)
plt.xlabel("Train Step")
plt.ylabel("Train Loss")

In [ ]:
# evaluate on 2x2 grid
n = 32
x = np.linspace(-1, 1, n)
y = x.copy()
xx, yy = np.meshgrid(x,y)
u = torch.tensor(np.stack([xx.ravel(), yy.ravel()], axis = -1), dtype= torch.float32)

In [ ]:
n_show = 5   # show ratemaps for first n_show nodes
cols = np.linspace(0,1, n_out)

def f(c):
    fig, ax = plt.subplots(n_show, n_show, figsize = (5, 5))
    context = c*torch.tensor(np.ones(len(u)), dtype = torch.float32)
    inputs = torch.cat((u, context[...,None]), dim = -1)
    p = model.spatial_representation(inputs)
    p = p.detach().numpy()

    max_ind = np.argmax(p, axis = 0)
    max_loc = u[max_ind]

    mask = np.mean(p, axis = 0) > 0 # mask out active units
    ax[0,0].set_title(f"Non-zero units: {mask.sum()}")

    ax[0,0].scatter(*max_loc.T, c = cols, cmap = "jet")
    ax[0,0].axis([-2, 2, -2, 2])
    ax[0,0].set_aspect("equal")

    count = 0
    for i in range(n_show):
        for j in range(n_show):
            if (i == 0) and (j == 0):
                pass
            else:
                ax[i,j].imshow(p[:,count].reshape((n,n)), cmap = "jet")
                ax[i,j].axis("off")
                count += 1
    plt.show()

c_slider = widgets.FloatSlider(
    value=0,
    min=-3.0,
    max=3.0,
    step=0.01,
    description='c:',
    readout_format='.2f')

widgets.interactive(f, c = c_slider)